### How to check if the ephemeris table has a sufficient time coverage for given MS data

In [1]:
from casatools import measures, table, quanta
me = measures()
qa = quanta()
tb = table()
import numpy as np

### Edit here
Specify MS, ephemeris table, telescope name, and field id

In [2]:
# MS data and the attached ephem table name
msname = 'europa.ms'
ephemtab = f'{msname}/FIELD/EPHEM0_Europa_59280.6.tab'
telescope = 'EVLA'
fieldid = '0'

In [3]:
tb.open(msname)
tb2 = tb.query(f'FIELD_ID={fieldid}', columns='TIME, FLAG_ROW')
tm = tb2.getcol('TIME')
flg = tb2.getcol('FLAG_ROW')
tb2.done()
tb.done()

True

Extract unique time stamps takeing account of flags

In [4]:
import numpy.ma as ma
masked_tm = ma.array(tm,mask=flg)
print(f'masked_tm: {len(masked_tm)}')
tm_compressed = masked_tm.compressed()
print(f'skipping masked ones: {len(tm_compressed)}')
tm_unique = np.unique(tm_compressed)
print(f'unique tm in tm_compressed {len(tm_unique)}')

masked_tm: 51622272
skipping masked ones: 31679757
unique tm in tm_compressed 1752


In [5]:
me.framecomet(ephemtab)
me.doframe(me.observatory(telescope))
me.doframe(me.epoch('utc', qa.quantity(tm_unique[0],'s')))
me.showframe()

'Frame: Epoch: 59280::14:04:31.0000 (TDB = 59280.6, UT1 = 59280.6, TT = 59280.6)\n       Position: [-1.60116e+06, -5.04199e+06, 3.55488e+06]\n        (Longitude = -1.87829 Latitude = 0.591675)\n       Direction: [0, 0, 1]\n        (J2000 = [0, 90] deg)\n       Europa comet between MJD 59280.6 and 59280.7'

In [6]:
nt = 0
tfailed =[]
for t in tm_unique:
    me.doframe(me.epoch('utc',qa.quantity(t,'s')))
    ret=me.measure(me.measure(me.direction('COMET'),'AZELGEO'),'ICRS')
    if ret=={}:
        tfailed.append(t)
    nt += 1    
print(f'Getting direction for {nt} timestamps, {len(tfailed)} failed.')

Getting direction for 1752 timestamps, 1 failed.


2024-03-20 22:03:08	WARN		Cannot convert due to missing frame information
2024-03-20 22:03:08	WARN	+	Cannot convert measure
2024-03-20 22:03:08	SEVERE		Illegal Measure record in MeasureHolder::fromRecord
2024-03-20 22:03:08	SEVERE	+	Non-measure type in measure conversion


In [7]:
print(tfailed)

[5121850019.0]


If the message above said 0 failed and tfailed list is empty, the ephemeris table is valid for the selected data. If no.
of the failed timestamps is not 0, one needs to either limit time range of the data to be used in tcleab by excluding the timestamps listed in the previous line or increase polyephem_tabtimestep during importasdm.